## Libraries

In [ ]:
!pip install openai-clip
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 7.0 MB/s eta 0:00:00
  Created wheel for openai-clip: filename=openai_clip-1.0.1-py3-none-any.whl size=1368605 sha256=f99276000b64a92a2b653f707e6748d9c7eb6f0c548d3a6ca22362c2d04506f6
  Stored in directory: /root/.cache/pip/wheels/08/77/8e/8d2f862df6bf7fb4e2007062d2cbaeae49862ec7b56d041229
Successfully built openai-clip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00


In [ ]:
import json
import string, os, re, pickle
import pandas as pd
from PIL import Image
import requests
from io import StringIO
from tqdm import tqdm
from datasets import Dataset

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import clip
from transformers import CLIPProcessor, CLIPModel, CLIPVisionConfig

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Create training and testing dataset

In [ ]:
image_path = "/content/drive/MyDrive/datasets/pokemon_png"
caption_path = "/content/drive/MyDrive/datasets/pokemon_caption.csv"
#read caption file
caption_df = pd.read_csv(caption_path)

#tidy up columns
caption_df.drop(caption_df.columns[[0,1,3]],axis=1,inplace=True)
caption_df.columns = ['pokedex','image','caption_1','caption_2']

#remove tag, carriage return, punctuation, leading/trailing space from caption
caption_df["caption_1"] = caption_df["caption_1"].str.replace('<p class="version-x active">\n', "")
caption_df["caption_1"] = caption_df["caption_1"].str.replace('\n                </p>', "")
caption_df["caption_2"] = caption_df["caption_2"].str.replace('<p class="version-y">\n', "")
caption_df["caption_2"] = caption_df["caption_2"].str.replace('\n                </p>', "")

caption_df["caption_1"] = caption_df["caption_1"].str.translate(str.maketrans('', '', string.punctuation))
caption_df["caption_2"] = caption_df["caption_2"].str.translate(str.maketrans('', '', string.punctuation))

caption_df["caption_1"] = caption_df["caption_1"].str.strip()
caption_df["caption_2"] = caption_df["caption_2"].str.strip()

#add filename column
caption_df['filename'] = caption_df["image"]

#concatenate caption 1 and 2
df1 = caption_df[['pokedex','filename','caption_1']].rename(columns={'caption_1': 'caption'})
df2 = caption_df[['pokedex','filename','caption_2']].rename(columns={'caption_2': 'caption'})
caption_df = pd.concat([df1,df2], ignore_index=True)

#drop caption rows with no image
caption_df.reset_index(drop=True, inplace=True)

#check
caption_df.head()

,pokedex,filename,caption
0,1,https://assets.pokemon.com/assets/cms2/img/pok...,For some time after its birth it uses the nutr...
1,2,https://assets.pokemon.com/assets/cms2/img/pok...,The more sunlight Ivysaur bathes in the more s...
2,3,https://assets.pokemon.com/assets/cms2/img/pok...,While it basks in the sun it can convert the l...
3,4,https://assets.pokemon.com/assets/cms2/img/pok...,The flame on its tail shows the strength of it...
4,5,https://assets.pokemon.com/assets/cms2/img/pok...,When it swings its burning tail the temperatur...


In [ ]:
class image_title_dataset():
    def __init__(self, list_image_path,list_txt):
        self.image_path = list_image_path
        self.title  = clip.tokenize(list_txt)

    def __len__(self):
        return len(self.title)

    def __getitem__(self, idx):
        image = preprocess(Image.open(requests.get(self.image_path[idx], stream=True).raw))
        title = self.title[idx]
        return image, title

In [ ]:
def gen():
    for index, row in caption_df.iterrows():
      yield {"caption":row["caption"], "filename":row["filename"]}

dataset = Dataset.from_generator(gen).shuffle(seed=123)
dataset = dataset.train_test_split(test_size=0.1, shuffle = False)

list_image_path = []
list_txt = []
for i in dataset["train"]:
  list_image_path.append(i["filename"])
  list_txt.append(i["caption"])

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset = image_title_dataset(list_image_path, list_txt)
train_dataloader = DataLoader(dataset, batch_size=100, shuffle=True)

## Pretrained models and parameters loading

In [ ]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

'cuda:0'

In [ ]:
#device = "cuda:0" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device, jit=False)

100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 136MiB/s]


In [ ]:
def convert_models_to_fp32(model):
    for p in model.parameters():
        p.data = p.data.float()
        p.grad.data = p.grad.data.float()

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5,betas=(0.9,0.98),eps=1e-6,weight_decay=0.2)
loss_img = nn.CrossEntropyLoss()
loss_txt = nn.CrossEntropyLoss()

## Model Training

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    pbar = tqdm(train_dataloader, total=len(train_dataloader))
    for batch in pbar:
        optimizer.zero_grad()

        images,texts = batch

        images= images.to(device)
        texts = texts.to(device)


        logits_per_image, logits_per_text = model(images, texts)


        ground_truth = torch.arange(len(images),dtype=torch.long,device=device)
        total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2


        total_loss.backward()
        convert_models_to_fp32(model)
        optimizer.step()
        clip.model.convert_weights(model)

        pbar.set_description(f"Epoch {epoch}/{num_epochs}, Loss: {total_loss.item():.4f}")

    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': total_loss,
        }, f"/content/drive/MyDrive/Pretrained Models/clip_hard_{epoch}.pt")

Epoch 9/10, Loss: 0.7168: 100%|██████████| 19/19 [03:17<00:00, 10.40s/it]
